In [52]:
import pandas as pd
import numpy as np
import requests
import urllib.parse
import googlemaps
import folium
from geographiclib.geodesic import Geodesic
from geographiclib.constants import Constants
geod = Geodesic.WGS84 
 
# Enter your API key here
api_key = "key"

gmaps = googlemaps.Client(key='key')


def return_lat_long(address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
    response = requests.get(url).json()
    #print(response)
    return (float(response[0]["lat"]) , float(response[0]["lon"]))

def get_directions(start,end,mode):
    return gmaps.directions(start, end,mode,alternatives= True)

def get_bearing(lat1, long1, lat2, long2):
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

def getEndpoint(lat1, lon1, bearing, d):
    geod = Geodesic(Constants.WGS84_a, Constants.WGS84_f)
    d = geod.Direct(lat1, lon1, bearing, d)
    return (d['lat2'], d['lon2'])

#print(getEndpoint(28.455556, -80.527778, 317.662819, 130.224835))
# (30.05352669918092, -82.21197985232848)


def weather_data(coordinates):
    
    api_key = "key"

    # base_url variable to store url
    base_url = "http://api.openweathermap.org/data/2.5/forecast?"

    # complete_url variable to store
    # complete url address
    complete_url = base_url + "lat=" + str(coordinates[0]) + "&lon=" + str(coordinates[1]) + "&appid=" + api_key 
    #print(complete_url)
    # get method of requests module
    # return response object
    response = requests.get(complete_url)

    # json method of response object
    # convert json format data into
    # python format data
    x = response.json()
    return x

def k_to_f(kelvin):
    return np.round(kelvin * 1.8 - 459.67,2)

def identify_closest_weather(x,date):
    df = pd.DataFrame(x['list'])
    df['difference'] = pd.to_datetime(df['dt_txt'])- pd.to_datetime(date)
    root_df = df[df['difference'] == df['difference'].min()] 
    root_df['main'].to_list()[0]['weather'] = root_df['weather'].to_list()[0][0]['main']
    return root_df['main'].to_list()[0]


def create_map(index):
    # use the response
    mls = total_coordinates
    points = [(i[0], i[1]) for i in mls[index]]
    m = folium.Map(zoom_start = 14,control_scale = True)
    # add marker for the start and ending points
    for point in [points[0], points[-1]]:
        folium.Marker(point).add_to(m)
    ##folium.Marker([lat, lon], popup=str(name)+': '+color+'-'+str(clname), icon=folium.Icon(color=color)).add_to(feature_group)
    snowflake_icon = folium.features.CustomIcon("http://clipart-library.com/images/pToArjn9c.png",icon_size=(28,30))
    for i,row in enumerate(location_data[index]):
        if weather_dict[index][i]['weather'] == "Rain": 
            folium.Marker((float(row[0]),float(row[1])), popup= return_req_weather_data(weather_dict,index,i),
                      icon=folium.Icon(icon="glyphicon-tint",color = "red")).add_to(m)
        elif weather_dict[index][i]['weather'] == "Snow": 
            folium.Marker((float(row[0]),float(row[1])),popup= return_req_weather_data(weather_dict,index,i),
                      icon=folium.Icon(icon="glyphicon-eye-close",color = "red")).add_to(m)
        elif weather_dict[index][i]['weather'] == "Clouds": 
            folium.Marker((float(row[0]),float(row[1])),popup= return_req_weather_data(weather_dict,index,i),
                      icon=folium.Icon(icon="glyphicon-cloud-upload",color = "green")).add_to(m)
        elif weather_dict[index][i]['weather'] == "Clear": 
            folium.Marker((float(row[0]),float(row[1])),popup= return_req_weather_data(weather_dict,index,i),
                      icon=folium.Icon(icon="glyphicon-certificate",color = "green")).add_to(m)
        
    # add the lines
    folium.PolyLine(points, weight=5, opacity=1).add_to(m)
    # create optimal zoom
    df = pd.DataFrame(mls[0]).rename(columns={0:'Lon', 1:'Lat'})[['Lat', 'Lon']]
    sw = df[['Lat', 'Lon']].min().values.tolist()
    ne = df[['Lat', 'Lon']].max().values.tolist()
    m.fit_bounds([sw, ne])
    return m

def return_req_weather_data(weather_dict,index,val):
    req_keys = ['temp','feels_like','humidity','weather']
    count = 0
    text = ''
    for key in req_keys:
        if count <2:
            text += " "+str(key)+":: " + str(k_to_f(weather_dict[index][val][key]))+" Degrees"
        else:    
            text += " "+ str(key) + ":: " +str(weather_dict[index][val][key])
        count+=1
    return text

class Weather_Indicator:
    
    def __init__(self,start,end,data,datetime):
        self.start = start
        self.end = end
        self.data = data
        self.datetime= datetime
        #print("Length of Data::: ",len(self.data))
        #print(self.data)
        self.route_coordinates = {}
        self.location_data = {}
        self.total_weather_data = {}
        
    def main(self):
        for index,route in enumerate(self.data):
            #print(route)
            self.route_coordinates[index],self.location_data[index],self.total_weather_data[index] = self.route_coordinates_indicator(route)
            #self.route_crimes[index] = self.identify_crimes(self.route_coordinates[index])
            #print("For the Route:: ", index," the total number of Crimes that happened on the way is:: ",self.route_crimes[index].shape[0],
        return self.route_coordinates,self.location_data,self.total_weather_data
                
    def route_coordinates_indicator(self,route):
        #print('##2', route)
        total_route_coordinates = []
        location_coordinates = []
        weather_info = []
        total_distance = 0
        temp_data = {}
        for leg in route['legs']:
            for step in leg['steps']:
                step_distance = step['distance']['value']
                start_loc_lat = step['start_location']['lat']
                start_loc_lng = step['start_location']['lng']
                end_loc_lat = step['end_location']['lat']
                end_loc_lng = step['end_location']['lng']
                #print(step_distance,start_loc_lat,end_loc_lat)
                difference_distance = 100000 - total_distance
                if (difference_distance > 0) & (step_distance >= difference_distance):
                    print('Req difference distance',difference_distance)
                    print("step_distance > required differnce distance value ", step_distance)
                    print("Calculating the bearing")
                    bearing = self.get_bearing(start_loc_lat,start_loc_lng,end_loc_lat,end_loc_lng)
                    total_route_coordinates.append((start_loc_lat,start_loc_lng))
                    for dist in range(int(np.floor((step_distance - difference_distance)/100000))+1):
                        print("Repeating the loop")
                        new_coordinates = self.getEndpoint(start_loc_lat,start_loc_lng,bearing,difference_distance)
                        print('New Coordinates generated::', new_coordinates)
                        weather_info.append(identify_closest_weather(weather_data(new_coordinates),self.datetime))
                        #print('Temperature at this location::', temp_data)
                        #weather_info.append(temp_data_1)
                        #crimes_data = crimes_data.append(return_number_events(df,'',19,new_coordinates,difference_distance))
                        start_loc_lat = new_coordinates[0]
                        start_loc_lng = new_coordinates[1]
                        location_coordinates.append((start_loc_lat,start_loc_lng))
                        if step_distance - difference_distance >= 100000:
                            difference_distance = 100000
                            print('Difference_distance updated to ',difference_distance)
                    print("Exited the loop, adjusting the total distance")
                    total_distance = (total_distance + step_distance)%100000
                    print("Total distance adjusted to:::",total_distance)
                elif (step_distance < difference_distance) & (difference_distance > 0):
                    print("Since step_distance is not long enough passing the loop")
                    total_distance += step_distance
                    print('Updated the total distance to :::', total_distance)
                elif (difference_distance <0):
                    print('This is unusual it should not happen')
                total_route_coordinates.append((end_loc_lat,end_loc_lng))
        return total_route_coordinates,location_coordinates,weather_info

    def get_bearing(self,lat1, long1, lat2, long2):
        brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
        return brng

    def getEndpoint(self,lat1, lon1, bearing, d):
        geod = Geodesic(Constants.WGS84_a, Constants.WGS84_f)
        d = geod.Direct(lat1, lon1, bearing, d)
        return (d['lat2'], d['lon2'])
    def k_to_f(self,kelvin):
        return np.round(kelvin * 1.8 - 459.67,2)
    

In [4]:
start = return_lat_long(address = '55 traveler Street, Boston, MA 02118')
end   = return_lat_long(address = 'Madison, WI 53590')

data = get_directions(start,end,'driving')

safety = Weather_Indicator(start,end,data)

total_coordinates,location_data,weather_dict = safety.main()

In [29]:
weather_dict

{0: [{'temp': 277.58,
   'feels_like': 273.3,
   'temp_min': 275.87,
   'temp_max': 277.58,
   'pressure': 1009,
   'sea_level': 1009,
   'grnd_level': 988,
   'humidity': 69,
   'temp_kf': 1.71,
   'weather': 'Clouds'},
  {'temp': 276.82,
   'feels_like': 272.19,
   'temp_min': 274.66,
   'temp_max': 276.82,
   'pressure': 1008,
   'sea_level': 1008,
   'grnd_level': 985,
   'humidity': 65,
   'temp_kf': 2.16,
   'weather': 'Snow'},
  {'temp': 275.77,
   'feels_like': 270.78,
   'temp_min': 273.44,
   'temp_max': 275.77,
   'pressure': 1008,
   'sea_level': 1008,
   'grnd_level': 985,
   'humidity': 69,
   'temp_kf': 2.33,
   'weather': 'Snow'},
  {'temp': 272.31,
   'feels_like': 265.93,
   'temp_min': 270.34,
   'temp_max': 272.31,
   'pressure': 1015,
   'sea_level': 1015,
   'grnd_level': 959,
   'humidity': 75,
   'temp_kf': 1.97,
   'weather': 'Snow'},
  {'temp': 273.7,
   'feels_like': 268.24,
   'temp_min': 272.48,
   'temp_max': 273.7,
   'pressure': 1013,
   'sea_level': 101

In [102]:
create_map(2)